In [1]:
import pandas as pd # https://pandas.pydata.org/docs/
import requests # https://docs.python-requests.org/en/latest/
import json # https://docs.python.org/3.9/library/json.html
import folium # http://python-visualization.github.io/folium/
import os # https://docs.python.org/3.9/library/os.html

os.chdir("C:/Users/kgilb/PycharmProjects/python-practice/python-practice/python-virtual-environments/wimlds/")

csvData = pd.read_csv("data/Water_Main_Breaks.csv")
csvData.head()

# run with command: jupyter notebook

,fullDate,location,leakClass,month,date,weekday,year,week,lon,lat,ObjectId
0,1.294963e+12,1205 W FAYETTE ST,0,1.0,14.0,NaN,2011.0,2.0,-76.175575,43.046733,1
1,1.294963e+12,1003 JAMESVILLE AVE,0,1.0,14.0,NaN,2011.0,2.0,-76.128918,43.016895,2
2,1.294963e+12,PALMER AVE & CHENEY ST,0,1.0,14.0,NaN,2011.0,2.0,-76.162257,43.030022,3
3,1.295136e+12,2100 E FAYETTE ST,0,1.0,16.0,NaN,2011.0,3.0,-76.113683,43.048158,4
4,1.295222e+12,206 BUTTERNUT ST,0,1.0,17.0,NaN,2011.0,3.0,-76.151936,43.057713,5


In [2]:
response = requests.get("https://services6.arcgis.com/bdPqSfflsdgFRVVM/arcgis/rest/services/Water_Main_Breaks/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
response.status_code

200

In [3]:
resData = response.json()
print(str(resData)[:1000]) # 1st 1000 characters of data2, same as (str(data2))[:1000]

{'objectIdFieldName': 'ObjectId', 'uniqueIdField': {'name': 'ObjectId', 'isSystemMaintained': True}, 'globalIdFieldName': '', 'geometryType': 'esriGeometryPoint', 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}, 'fields': [{'name': 'fullDate', 'type': 'esriFieldTypeDate', 'alias': 'fullDate', 'sqlType': 'sqlTypeTimestamp2', 'length': 8, 'domain': None, 'defaultValue': None}, {'name': 'location', 'type': 'esriFieldTypeString', 'alias': 'location', 'sqlType': 'sqlTypeNVarchar', 'length': 256, 'domain': None, 'defaultValue': None}, {'name': 'leakClass', 'type': 'esriFieldTypeInteger', 'alias': 'leakClass', 'sqlType': 'sqlTypeInteger', 'domain': None, 'defaultValue': None}, {'name': 'month', 'type': 'esriFieldTypeString', 'alias': 'month', 'sqlType': 'sqlTypeNVarchar', 'length': 256, 'domain': None, 'defaultValue': None}, {'name': 'date', 'type': 'esriFieldTypeString', 'alias': 'date', 'sqlType': 'sqlTypeNVarchar', 'length': 256, 'domain': None, 'defaultValue': None}, {'name': 'week

In [4]:
# overwrite data, won't be using in rest of code and don't want it to be available
resData = pd.io.json.json_normalize(resData["features"])

# use attributes to find (.FID) each column and rename it
resData.rename(columns={
    'attributes.FID': 'FID',
    'attributes.TNT_NAME': 'TNT_NAME',
    'attributes.date': 'date',
    'attributes.fullDate': 'fullDate',
    'attributes.lat': 'lat',
    'attributes.leakClasss': 'leakClass',
    'attributes.location': 'location',
    'attributes.lon': 'lon',
    'attributes.month': 'month',
    'attributes.week': 'week',
    'attributes.weekday': 'weekday',
    'attributes.year': 'year',
    'geometry.x': 'x',
    'geometry.y': 'y'}, inplace=True)
resData.head()

C:\Users\kgilb\AppData\Local\Temp/ipykernel_16840/4019065537.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  resData = pd.io.json.json_normalize(resData["features"])


,fullDate,location,attributes.leakClass,month,date,weekday,year,week,lon,lat,attributes.ObjectId,x,y
0,1294963200000,1205 W FAYETTE ST,0,1.0,14.0,None,2011.0,2.0,-76.175575,43.046733,1,-76.175575,43.046733
1,1294963200000,1003 JAMESVILLE AVE,0,1.0,14.0,None,2011.0,2.0,-76.128918,43.016895,2,-76.128918,43.016895
2,1294963200000,PALMER AVE & CHENEY ST,0,1.0,14.0,None,2011.0,2.0,-76.162257,43.030022,3,-76.162257,43.030022
3,1295136000000,2100 E FAYETTE ST,0,1.0,16.0,None,2011.0,3.0,-76.113683,43.048158,4,-76.113683,43.048158
4,1295222400000,206 BUTTERNUT ST,0,1.0,17.0,None,2011.0,3.0,-76.151936,43.057713,5,-76.151936,43.057713
